In [1]:
import numpy as np
import ctypes

def coords_to_indices(coords, width):
    return coords[:, 1] * width + coords[:, 0]

def indices_to_coords(indices, width):
    y = indices // width
    x = indices % width
    return np.vstack((x, y)).T

a = np.array([(3, 6), (3, 7), (3, 8), (1, 5), (5, 5), (0, 9)])
b = np.array([(3, 8), (5, 5), (3, 4), (3, 9), (1, 7), (0, 7), (2, 0)])

w = 10

list1_indices = coords_to_indices(a, w).astype(np.int32)
list2_indices = coords_to_indices(b, w).astype(np.int32)
print(list1_indices, list2_indices)
lib = ctypes.CDLL('./findIntersection.so')

lib.launchFindIntersection.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.int32),  # list1
    np.ctypeslib.ndpointer(dtype=np.int32),  # list2
    ctypes.c_int,  # list1Size
    ctypes.c_int,  # list2Size
    ctypes.c_int,  # width
    np.ctypeslib.ndpointer(dtype=np.int32),  # resultSizes
    np.ctypeslib.ndpointer(dtype=np.int32),  # upResult
    np.ctypeslib.ndpointer(dtype=np.int32),  # downResult
    np.ctypeslib.ndpointer(dtype=np.int32),  # leftResult
    np.ctypeslib.ndpointer(dtype=np.int32)   # rightResult
]

result_size_array = np.zeros(4, dtype=np.int32)
up_result = np.zeros(len(list1_indices), dtype=np.int32)
down_result = np.zeros(len(list1_indices), dtype=np.int32)
left_result = np.zeros(len(list1_indices), dtype=np.int32)
right_result = np.zeros(len(list1_indices), dtype=np.int32)

lib.launchFindIntersection(
    list1_indices,
    list2_indices,
    len(list1_indices),
    len(list2_indices),
    w,
    result_size_array,
    up_result,
    down_result,
    left_result,
    right_result
)
print("Length of results [U D L R]",result_size_array)
up_intersection = indices_to_coords(up_result[:result_size_array[0]], w)
down_intersection = indices_to_coords(down_result[:result_size_array[1]], w)
left_intersection = indices_to_coords(left_result[:result_size_array[2]], w)
right_intersection = indices_to_coords(right_result[:result_size_array[3]], w)

print("Up Intersection:\n", up_intersection)
print("Down Intersection:\n", down_intersection)
print("Left Intersection:\n", left_intersection)
print("Right Intersection:\n", right_intersection)


[63 73 83 51 55 90] [83 55 43 93 71 70  2]
L1: 
53 57 65 75 85 92 
0 0 0 0 0 0 
------------------
L1: 
49 53 61 71 81 88 
71 0 0 0 0 0 
------------------
L1: 
71 75 83 93 103 110 
71 83 93 0 0 0 
------------------
L1: 
31 35 43 53 63 70 
43 70 0 0 0 0 
------------------
Length of results [U D L R] [0 1 3 2]
Up Intersection:
 []
Down Intersection:
 [[1 7]]
Left Intersection:
 [[1 7]
 [3 8]
 [3 9]]
Right Intersection:
 [[3 4]
 [0 7]]
